<a href="https://colab.research.google.com/github/mapsguy/programming-gemini/blob/main/generating_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-genai google-cloud-aiplatform Pillow

In [3]:
#import the genai library
from google import genai

In [4]:
#step 2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_API_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [14]:
from google import genai
from google.genai import types
from PIL import Image
import io
from io import BytesIO
import base64

prompt = "A friendly robot waving hello, with a speech bubble saying 'Greetings!'"

gen_config = types.GenerateContentConfig(
    response_modalities=['TEXT', 'IMAGE']
)

response = client.models.generate_content(
    model='gemini-2.0-flash-exp-image-generation',
    contents=prompt,
    config=gen_config
)

for part in response.candidates[0].content.parts:
  if part.text is not None:
    print(part.text)
  elif part.inline_data is not None:
    image = Image.open(BytesIO((part.inline_data.data)))
    image.save('gemini-image.png')
    print("Image generated and saved.")

I will generate an image of a cheerful, cartoon-style robot with rounded features and bright, friendly eyes. The robot will be mid-wave, with one of its metallic arms raised in a welcoming gesture. A speech bubble will be positioned near its head, clearly displaying the handwritten-style text "Greetings!". The background will be simple and uncluttered to keep the focus on the robot.

Image generated and saved.


In [16]:
#REST API example for image generation

%%bash

curl -s -X POST \
  "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp-image-generation:generateContent?key=$GEMINI_API_KEY" \
  -H "Content-Type: application/json" \
  -d '{
    "contents": [{
      "parts": [
        {"text": "Create a friendly robot waving hello, with a speech bubble saying 'Greetings!'"}
      ]
    }],
    "generationConfig":{"responseModalities":["TEXT","IMAGE"]}
  }' \
  | grep -o '"data": "[^"]*"' \
  | cut -d'"' -f4 \
  | base64 --decode > gemini-image.png

In [ ]:
#I can't run this lol

#High quality image generation with Imagen 3

from google.genai import types

#fine-grained control over image generation

model_name = "imagen-3.0-generate-002" # Or other available Imagen 3 model

gen_config = types.GenerateImagesConfig(
    aspect_ratio="1:1",
    number_of_images=1,
    negative_prompts="Blurry, Cartoon",
    seed=42,
    person_generation="false" # Set to "true" to allow generating people.
)

response = client.models.generate_images(
    model=model_name,
    config=gen_config,
    prompt="A photorealistic image of a red panda playing a tiny piano in a lush forest.",

)

for generated_image in response.generated_images:
  image = Image.open(BytesIO(generated_image.image.image_bytes))
  image.save('imagen-image.png')
